In [1]:
!pip install numpy sklearn pandas gensim
!pip install sastrawi nltk

In [0]:
import pandas as pd
import numpy as np
import re
from string import *
import csv
from nltk import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import json
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [3]:
from google.colab import drive
drive.mount('/dataset')
#4/aAD9sbW_SBavNLpLBxoVoh1InxyMiPmW__PMe8T03sb5qJgdsPTM0IY

Mounted at /dataset


In [0]:
with open('/dataset/My Drive/dataset/pos.txt', 'r') as f:
    pos = f.read().splitlines()
    
with open('/dataset/My Drive/dataset/neg.txt', 'r') as f:
    neg = f.read().splitlines()

In [0]:
neg_label = []
pos_label = []
for i in range(len(neg)):
  neg_label.append('negative')
  
for i in range(len(pos)):
  pos_label.append('positive')

In [0]:
x_pos_train, x_pos_test = train_test_split(pos, test_size=0.2)
x_neg_train, x_neg_test = train_test_split(neg, test_size=0.2)
y_pos_train, y_pos_test = train_test_split(pos_label, test_size=0.2)
y_neg_train, y_neg_test = train_test_split(neg_label, test_size=0.2)

x_train = x_pos_train + x_neg_train
y_train = y_pos_train + y_neg_train
x_test = x_pos_test + x_neg_test
y_test = y_pos_test + y_neg_test

In [7]:
!python
# nltk.download('punkt')
# nltk.download('stopwords')

Python 2.7.14 (default, Sep 23 2017, 22:06:14) 
[GCC 7.2.0] on linux2
Type "help", "copyright", "credits" or "license" for more information.
>>> import nltk
>>> nltk.download('punkt')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
True
>>> nltk.download('stopwords')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
True
>>> exit()


In [0]:
def preprocessing(data):
  data = [line.lower() for line in data]
  
  data = [''.join(c for c in s if c not in punctuation) for s in data]
  
  tokenized = []
  for line in data:
      result = word_tokenize(line)
      #print(result)
      tokenized.append(result)
  
  stop_word = tokenized[:]
  stopWords = stopwords.words('indonesian')
  for i in range(len(stop_word)):
      words = stop_word[i]
      wordsFiltered =[]
      for w in words:
          if w not in stopWords:
              wordsFiltered.append(w)
          stop_word[i] = wordsFiltered
          
  #declare list stemm dari stopword
  stem_word = stop_word[:]
  # create stemmer
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  # stemming process
  for i in range(len(stem_word)):
      for j in range(len(stem_word[i])):
          stem_word[i][j] = stemmer.stem(stem_word[i][j])
  
  return stem_word

hasil = preprocessing(x_train)

In [9]:
print hasil

[['lancar'], ['lupa', 'mema', 'afkan', 'harap', 'komponen', 'bangsa', 'ambil', 'nilai', 'positif', 'tragedi', 'tragedi', 'bencana', 'manusia', 'ulang', 'mana'], ['mantap', 'berani', 'diri', 'usaha', 'indonesia'], ['pilih'], ['kuat', 'rupiah', 'pekan', 'salah', 'satu', 'picu', 'data', 'ekonomi', 'as', 'rilis', 'positif', 'kemarin'], ['bidang', 'sehat', 'prioritas', 'bangun', 'kabupaten', 'bandung'], ['maret', 'tuju', 'tuju', 'benchmark'], ['kalimantan', 'anggap', 'cocok', 'kembang', 'nuklir', 'karakteristik', 'bebas', 'gempa', 'banding', 'jawa'], ['istimewa', 'harga', 'klub', 'hebat'], ['pamer', 'harap', 'oleh', 'transaksi', 'beli', 'kerjasama', 'bisnis', 'usaha', 'uae', 'negaranegara'], ['dunk', 'tambah', 'apa'], ['bisnis', 'kondisi', 'antisipasi', 'beban', 'percaya', 'kuat', 'pasar', 'efektif', 'sustainable', 'tentu', 'suku', 'bunga'], ['bangsa', 'cepat', 'sadar'], ['serah', 'abdi', 'hukum'], ['objek', 'wisata', 'duludulu', 'putus', 'mudah', 'wisatawan', 'praktis'], ['selenggara', 'ha

In [10]:
joinedtogether = []
for i in range(len(hasil)):
    joinedtogether.append(" ".join(hasil[i]))
print(joinedtogether)

['lancar', 'lupa mema afkan harap komponen bangsa ambil nilai positif tragedi tragedi bencana manusia ulang mana', 'mantap berani diri usaha indonesia', 'pilih', 'kuat rupiah pekan salah satu picu data ekonomi as rilis positif kemarin', 'bidang sehat prioritas bangun kabupaten bandung', 'maret tuju tuju benchmark', 'kalimantan anggap cocok kembang nuklir karakteristik bebas gempa banding jawa', 'istimewa harga klub hebat', 'pamer harap oleh transaksi beli kerjasama bisnis usaha uae negaranegara', 'dunk tambah apa', 'bisnis kondisi antisipasi beban percaya kuat pasar efektif sustainable tentu suku bunga', 'bangsa cepat sadar', 'serah abdi hukum', 'objek wisata duludulu putus mudah wisatawan praktis', 'selenggara haji umrah kerja mitra bina layan lindung jamaah', 'tumbuh investasi kota depok imbang sebar bangun', 'cinta negeri bangun kampung', 'haiban hajid ssos wartawan selasela lomba serta lomba sedia puluh hadiah', 'jalan asimilasi rutan ariel kerja sesuai atur laku', 'kerja uni eropa

In [0]:
vectorizer = TfidfVectorizer(min_df = 1, max_df = 0.4, sublinear_tf = True, use_idf = True, smooth_idf=False)

def tfidf_train(data):
  data = vectorizer.fit_transform(data)
  data = data.toarray()
  return data

def tfidf_test(data):
  data = vectorizer.transform(data)
  data = data.toarray()
  return data

data_train = tfidf_train(joinedtogether)
data_test = tfidf_test(x_test)

In [0]:
def train_random_forest(data_train, data_test, tree=100, method='entropy'):
  forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion=method,
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=tree, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
  model = forest.fit( data_train, data_test )
  return model

def train_dtree(data_train, data_test):
  dtree = DecisionTreeClassifier(random_state=0, max_depth=2)
  model = dtree.fit( data_train, data_test )
  return model

In [0]:
#data_train = vectorizer.fit_transform(joinedtogether)
#data_train = data_train.toarray()

#forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
#            max_depth=4, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
#model = forest.fit(data_train, y_train)

rf_model = train_random_forest(data_train, y_train)
dt_model = train_dtree(data_train, y_train)

In [0]:
prediction = dict()
prediction['RandomForest'] = rf_model.predict(data_test)
prediction['DecisionTree'] = dt_model.predict(data_test)

In [16]:
print metrics.classification_report(y_test, prediction['RandomForest'])
print metrics.classification_report(y_test, prediction['DecisionTree'])

             precision    recall  f1-score   support

   negative       0.59      0.82      0.68       200
   positive       0.70      0.42      0.53       200

avg / total       0.64      0.62      0.60       400

             precision    recall  f1-score   support

   negative       0.50      1.00      0.67       200
   positive       0.00      0.00      0.00       200

avg / total       0.25      0.50      0.33       400



/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
print "Akurasi model random forest : {}%".format(metrics.accuracy_score(y_test, prediction['RandomForest'])*100)
print "Akurasi model decision tree : {}%".format(metrics.accuracy_score(y_test, prediction['DecisionTree'])*100)

Akurasi model random forest : 62.0%
Akurasi model decision tree : 50.0%


In [45]:
from __future__ import division
cnf_forest = metrics.confusion_matrix(y_test, prediction['RandomForest'])
cnf_tree = metrics.confusion_matrix(y_test, prediction['DecisionTree'])
print "Confusion matrix random forest : \n{}".format(cnf_forest[0][0]/(cnf_forest[0][0]+cnf_forest[0][1])*100)
print "Confusion matrix decision tree : \n{}".format(cnf_tree[0][0]/(cnf_tree[0][0]+cnf_tree[0][1])*100)

Confusion matrix random forest : 
82.0
Confusion matrix decision tree : 
100.0


In [0]:
def predict_sample(model, sample):
    sample_counts = vectorizer.transform([sample])
    sample_tfidf = sample_counts.toarray()
    result = model.predict(sample_tfidf)[0]
    prob = model.predict_proba(sample_tfidf)[0]
    print 'Hasil prediksi {}, Positive : {}, Negative : {}'.format(result, prob[1], prob[0])

In [47]:
# Random Forest
predict_sample(rf_model, "Direktur dan Chairman Oldham secara khusus berterima kasih kepada Liverpool FC dan kepolisian Merseyside atas keseriusan mereka mengusut kasus Adeyemi. Komunikasi yang baik antar kedua klub serta para pemain sangat membantu penyelesaian kasus tersebut")
predict_sample(rf_model, "Eksekusi hukuman mati Ruyati binti Satubi merupakan salah satu contohnya, meskipun sejak awal perwakilan RI telah mengawal proses hukumnya")

Hasil prediksi positive, Positive : 0.512066185177, Negative : 0.487933814823
Hasil prediksi negative, Positive : 0.488292317178, Negative : 0.511707682822


In [48]:
# Decision Tree
predict_sample(dt_model, "Direktur dan Chairman Oldham secara khusus berterima kasih kepada Liverpool FC dan kepolisian Merseyside atas keseriusan mereka mengusut kasus Adeyemi. Komunikasi yang baik antar kedua klub serta para pemain sangat membantu penyelesaian kasus tersebut")
predict_sample(dt_model, "Eksekusi hukuman mati Ruyati binti Satubi merupakan salah satu contohnya, meskipun sejak awal perwakilan RI telah mengawal proses hukumnya")

Hasil prediksi negative, Positive : 0.493983957219, Negative : 0.506016042781
Hasil prediksi negative, Positive : 0.493983957219, Negative : 0.506016042781


In [49]:
from sklearn.externals import joblib

joblib.dump(vectorizer.vocabulary_, '/dataset/My Drive/dataset/vectorizer_vocab.pkl')

joblib.dump(rf_model, '/dataset/My Drive/dataset/randomforest.pkl')

['/dataset/My Drive/dataset/randomforest.pkl']

In [0]:
rforest_model = joblib.load('/dataset/My Drive/dataset/randomforest.pkl')

In [51]:
predict_sample(rforest_model, "Direktur dan Chairman Oldham secara khusus berterima kasih kepada Liverpool FC dan kepolisian Merseyside atas keseriusan mereka mengusut kasus Adeyemi. Komunikasi yang baik antar kedua klub serta para pemain sangat membantu penyelesaian kasus tersebut")

Hasil prediksi positive, Positive : 0.512066185177, Negative : 0.487933814823
